In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum emission

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * 50 / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
gridSize = DiscreteParameter('gridSize', domain=[3, 4, 5, 6, 7, 8, 9, 10])
edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=8, max_value=20)  # 28.8 km/h to 72 km/h
numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1, 2, 3])
accel = ContinuousParameter('accel', 1., 3.)

parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, numberOfLanes, accel])
#parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
#init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/emissions_200_init_points.pkl', 'rb'))

design = RandomDesign(parameter_space) 
num_init_points = 250
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions(init_X)

[[10.         14.3941175   3.          2.40989451]
 [ 6.         16.50534565  2.          2.32691315]
 [ 3.          9.48907108  1.          1.31720028]
 [ 7.         18.89699467  2.          1.51799023]
 [ 3.         17.32917171  2.          2.2122406 ]
 [ 5.          9.46855122  1.          2.13461924]
 [ 7.          9.06985734  1.          2.15313339]
 [ 9.         15.68064139  3.          1.78260641]
 [ 3.         16.26846333  1.          1.27120255]
 [ 4.         16.74902719  1.          2.89326729]
 [ 9.         17.14986084  3.          2.45904413]
 [ 9.         12.70028713  3.          2.65184914]
 [ 7.         19.1526701   1.          1.73478691]
 [ 4.         18.58545858  1.          1.06861139]
 [ 4.         10.36490701  3.          1.32492648]
 [ 3.         13.40256781  1.          1.22098735]
 [ 4.         13.67923539  1.          1.58760885]
 [ 7.         12.58067027  1.          2.63925179]
 [ 7.         10.17494264  3.          1.90317068]
 [ 4.         12.47105278  1.  

 Retrying in 1 seconds

Evaluating input: 2 of 250

 Retrying in 1 seconds

Evaluating input: 3 of 250

 Retrying in 1 seconds

Evaluating input: 4 of 250

 Retrying in 1 seconds

Evaluating input: 5 of 250

 Retrying in 1 seconds

Evaluating input: 6 of 250

 Retrying in 1 seconds

Evaluating input: 7 of 250

 Retrying in 1 seconds

Evaluating input: 8 of 250

 Retrying in 1 seconds

Evaluating input: 9 of 250

 Retrying in 1 seconds

Evaluating input: 10 of 250

 Retrying in 1 seconds

Evaluating input: 11 of 250

 Retrying in 1 seconds

Evaluating input: 12 of 250

 Retrying in 1 seconds

Evaluating input: 13 of 250

 Retrying in 1 seconds

Evaluating input: 14 of 250

 Retrying in 1 seconds

Evaluating input: 15 of 250

 Retrying in 1 seconds

Evaluating input: 16 of 250

 Retrying in 1 seconds

Evaluating input: 17 of 250

 Retrying in 1 seconds

Evaluating input: 18 of 250

 Retrying in 1 seconds

Evaluating input: 19 of 250

 Retrying in 1 seconds

Evaluating input: 20 of 250

 

 Retrying in 1 seconds

Evaluating input: 156 of 250

 Retrying in 1 seconds

Evaluating input: 157 of 250

 Retrying in 1 seconds

Evaluating input: 158 of 250

 Retrying in 1 seconds

Evaluating input: 159 of 250

 Retrying in 1 seconds

Evaluating input: 160 of 250

 Retrying in 1 seconds

Evaluating input: 161 of 250

 Retrying in 1 seconds

Evaluating input: 162 of 250

 Retrying in 1 seconds

Evaluating input: 163 of 250

 Retrying in 1 seconds

Evaluating input: 164 of 250

 Retrying in 1 seconds

Evaluating input: 165 of 250

 Retrying in 1 seconds

Evaluating input: 166 of 250

 Retrying in 1 seconds

Evaluating input: 167 of 250

 Retrying in 1 seconds

Evaluating input: 168 of 250

 Retrying in 1 seconds

Evaluating input: 169 of 250

 Retrying in 1 seconds

Evaluating input: 170 of 250

 Retrying in 1 seconds

Evaluating input: 171 of 250

 Retrying in 1 seconds

Evaluating input: 172 of 250

 Retrying in 1 seconds

Evaluating input: 173 of 250

 Retrying in 1 seconds

Eval

To then fit a GP to these points

In [7]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = 1119.4230495971256


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 2/5, f = 1042.0469196786814
Optimization restart 3/5, f = 1042.0469196080724
Optimization restart 4/5, f = 1042.0469196074794
Optimization restart 5/5, f = 1042.0469196074744


### Experimentally designed emulator

We also load the emulator obtained during experimental design:

In [8]:
emukit_model_exp_design = pickle.load(open('experimental_design/models/model_variance_20_init_points_250_loops_co2.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [9]:
n_iter_bo = 40

### Acquisition functions

In [10]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [11]:
acquisition_exp_design = ExpectedImprovement(emukit_model_exp_design)

### Optimisation loop

In [ ]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 1042.0469196074744


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1042.0469196074987
Optimization restart 3/5, f = 1042.38086665382
Optimization restart 4/5, f = 1042.0469196942913
Optimization restart 5/5, f = 1042.0469196138695
[[10.  8.  1.  1.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1046.0170961130543
Optimization restart 2/5, f = 1046.0110271343656
Optimization restart 3/5, f = 1046.011027139721
Optimization restart 4/5, f = 1046.0110271630924
Optimization restart 5/5, f = 1046.0110281846978
[[10.  8.  1.  1.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


...

Optimization restart 1/5, f = 1165.9988909600495
Optimization restart 2/5, f = 1165.9988861589777
Optimization restart 3/5, f = 1165.9988861576335
Optimization restart 4/5, f = 1165.9988867266168
Optimization restart 5/5, f = 1165.9988861715567
[[10.  8.  3.  1.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1169.4471482019808
Optimization restart 2/5, f = 1169.4471474534537
Optimization restart 3/5, f = 1169.447147453525
Optimization restart 4/5, f = 1169.4471474534994
Optimization restart 5/5, f = 1169.4471474534776
[[10.  8.  3.  1.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1172.882864330274
Optimization restart 2/5, f = 1172.8828632264867
Optimization restart 3/5, f = 1172.8828632265713
Optimization restart 4/5, f = 1172.8828632264003


In [ ]:
bo_exp_design = BayesianOptimizationLoop(parameter_space, emukit_model_exp_design, acquisition=acquisition_exp_design)
bo_exp_design.run_loop(user_function_emissions, n_iter_bo)

## Results

### Results on the random initialised emulator

In [ ]:
results_random_init = bo_random_init.get_results()

In [ ]:
results_random_init.minimum_location

In [ ]:
results_random_init.minimum_value*100

### Results on the experimentally designed emulator

In [ ]:
results_exp_design = bo_exp_design.get_results()

In [ ]:
results_exp_design.minimum_location

In [ ]:
results_exp_design.minimum_value*100